| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`Where` |Returns a subset of elements that satisfy a given condition |WHERE
|`Take` |Returns the first count elements and discards the rest |WHERE ROW_NUMBER()...<br>or TOP n subquery
|`Skip` |Ignores the first count elements and returns the rest |WHERE ROW_NUMBER()...<br>or NOT IN (SELECT TOP n...)
|`TakeLast` |Takes only the last count elements |Exception thrown
|`SkipLast` |Takes only the last count elements |Exception thrown
|`TakeWhile` |Emits elements from the input sequence until the<br>predicate is false |Exception thrown
|`SkipWhile` |Ignores elements from the input sequence until<br>the predicate is false, and then emits the rest |Exception thrown
|`Distinct`,<br>`DistinctBy` |Returns a sequence that excludes duplicates |SELECT DISTINCT...

متدهای فیلترینگ، همان تعداد یا تعداد کمتری المان بر می گردانند، در ضمن هیچ تغییری هم در المانها بوجود نمی آورند

### Where

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Predicate |TSource => bool or (TSource,int) => bool

In [ ]:
public static IEnumerable<TSource> Where<TSource>
(this IEnumerable<TSource> source, Func <TSource, bool> predicate)
{
    foreach (TSource element in source)
        if (predicate (element))
            yield return element;
}

In [ ]:
string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };
IEnumerable<string> fluentQuery = names.Where (name => name.EndsWith ("y"));

IEnumerable<string> query = 
    from n in names
    where n.EndsWith ("y")
    select n;

A ***where*** clause can appear `more than once` in a query and be interspersed with `let`, `orderby`, and `join` clauses

In [ ]:
string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };

IEnumerable<string> query =
    from n in names
    where n.Length > 3
    let u = n.ToUpper()
    where u.EndsWith ("Y")
    select u;

***Indexed filtering***

In [ ]:
string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };

IEnumerable<string> query = names.Where ((n, i) => i % 2 == 0);

>An `exception is thrown` if you use indexed filtering in `EF Core`.

***SQL LIKE comparisons in EF Core***

The following methods on string translate to `SQL’s LIKE` operator:  
`Contains`, `StartsWith`, `EndsWith`

`c.Name.Contains ("abc")` => `customer.Name LIKE '%abc%'`

to compare against `another column`   
  
where `EF.Functions.Like (c.Description, "%" + c.Name + "%")`

***< and > string comparisons in EF Core***

You can perform order comparison on strings with string’s `CompareTo` method;

`dbContext.Purchases.Where (p => p.Description.CompareTo ("C") < 0)`  
خریدهایی که توضیحات آنها با حروف A , B شروع می شوند

***WHERE x IN (…, …, …) in EF Core***

In [ ]:
string[] chosenOnes = { "Tom", "Jay" };

var query =
from c in dbContext.Customers
where chosenOnes.Contains (c.Name)
select c;

//WHERE customer.Name IN ("Tom", "Jay")

### Take, TakeLast, Skip, and SkipLast

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Number of elements to take or skip |int

In [ ]:
//The following returns the first 20:
IQueryable<Book> query = dbContext.Books
.Where (b => b.Title.Contains ("mercury"))
.OrderBy (b => b.Title)
.Take (20);

//The next query returns books 21 to 40:
IQueryable<Book> query1 = dbContext.Books
.Where (b => b.Title.Contains ("mercury"))
.OrderBy (b => b.Title)
.Skip (20).Take (20);

***EF Core*** translates `Take` and `Skip` to the `ROW_NUMBER` function in `SQL Server 2005`, or a `TOP n` subquery in `earlier versions of SQL Server`.

The `TakeLast` and `SkipLast` methods **take** or **skip** the `last n elements`.

`Take(5..)` is equivalent to `Skip(5)`  
  
`Take(..^5)` is equivalent to `SkipLast(5)`.

### TakeWhile and SkipWhile

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Predicate |TSource => bool or (TSource,int) => bool

In [ ]:
int[] numbers = { 3, 5, 2, 234, 4, 1 };
var takeWhileSmall = numbers.TakeWhile (n => n < 100); // { 3, 5, 2 }

In [ ]:
int[] numbers = { 3, 5, 2, 234, 4, 1 };
var skipWhileSmall = numbers.SkipWhile (n => n < 100); // { 234, 4, 1 }

***TakeWhile*** and ***SkipWhile*** have `no translation to SQL` and **throw an exception** if used in an `EF Core` query.

### Distinct and DistinctBy

In [ ]:
//Distinct returns the input sequence, stripped of duplicates.
char[] distinctLetters = "HelloWorld".Distinct().ToArray();
string s = new string (distinctLetters);   // HeloWrd

In [1]:
/*The DistinctBy method was introduced in .NET 6 and lets you specify a key
selector to be applied before performing equality comparison.*/

new[] { 1.0, 1.1, 2.0, 2.1, 3.0, 3.1 }
.DistinctBy (n => Math.Round (n, 0));  //{1,2,3}